In [1]:
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime
from matplotlib import pyplot as plt

# Aggregating data

In [ ]:
df_ged =  pd.read_csv('GED_Gaul.csv', encoding='latin-1')



d_parser = lambda x: pd.datetime.strptime(x, '%d %B %Y')#cuz date format in 'ACLED' file is different, we need to change the format of date 


df_acld = pd.read_csv("AC_gaul.csv", parse_dates = ['event_date'], date_parser = d_parser, encoding='latin-1' )

In [ ]:



Country_list=['Algeria','Angola','Burkina Faso','Burundi','Cameroon','Central African Republic','Chad',
            'Congo','Djibouti','Eritrea','Ethiopia','Gambia','Ghana','Guinea',
            'Guinea-Bissau','Kenya','Liberia','Libya','Algeria',
            'Mauritania','Morocco','Mozambique','Namibia','Niger','Nigeria','Rwanda','Senegal','Sierra Leone',
            'Mali','Somalia','South Africa','South Sudan','Sudan','Togo','Tunisia','Uganda','Zambia','Zimbabwe (Rhodesia)']

# collect african countries from our dataframe

df_ged_afc = df_ged[df_ged.adm0_name.isin(Country_list)]

#remove unnecessary columns from our two dataframes

df_ged_afc = df_ged_afc.drop(['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23',
            'Unnamed: 24','Unnamed: 26', 'Unnamed: 27','Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63'], axis=1)

df_acld.rename(columns = {'adm0_name':'adm0', 'adm1_name': 'adm1', 'adm2_name': 'adm2'}, inplace= True)
df_acld['month'] = df_acld['event_date'].dt.month


#devide "date" column to month and year
df_ged_afc[['year','month','day']]= df_ged_afc['date_start'].str.split('-', expand=True)


#create column 'number of event' where "best"(best estimate of fatalities in GED ) and fatalities in AClEd, are available

df_ged_afc["number_event"] = np.where(df_ged_afc["best"] >0 , 1, 0)

df_acld["number_event"] = np.where(df_acld["fatalities"] >0 , 1, 0)


#then aggregate them on number of fatalities and number of event

agg_ged = df_ged_afc.groupby(['year','month','adm0','adm1', 'adm2', 'latitude','longitude']).agg({'best':['sum'], 'number_event':['sum']})

df_acld_agg = df_acld.groupby(['year','month','adm0','adm1', 'adm2', 'latitude','longitude']).agg({'fatalities':['sum'], 'number_event':['sum']})


# calculating inflation

In [ ]:
Country_list=['Algeria','Angola','Burkina Faso','Burundi','Cameroon','Central African Republic','Chad',
            'Congo','Djibouti','Eritrea','Ethiopia','Gambia','Ghana','Guinea',
            'Guinea-Bissau','Kenya','Liberia','Libya','Algeria',
            'Mauritania','Morocco','Mozambique','Namibia','Niger','Nigeria','Rwanda','Senegal','Sierra Leone',
            'Mali','Somalia','South Africa','South Sudan','Sudan','Togo','Tunisia','Uganda','Zambia','Zimbabwe (Rhodesia)']




df =  pd.read_csv('match_price_loc.csv')#loading('match_price_loc.csv')dataframe that has every market with its location and prices for each product(CSV file is available)

#to calculate amount of inflation for each product in each markets for all adm2 in all countries

for i in Country_list:
    df_adm0 = df[df['adm0_name']==i]

#aggregating data frame according to price 
    df_agg = df_adm0.groupby(['mp_year', 'mp_month','adm0_name','adm1_name','admin2', 'market', 'cm_name']).agg({'mp_price':['sum']})
    




    for j in df_agg.adm2_name.unique():
    
    
    
        df_ag_adm2 = df_agg[df_agg['adm2_name']==j]
        df_ag_adm2 = df_ag_adm2.groupby(['year', 'month','adm0_name','adm1_name','adm2_name', 'market', 'cm_name']).agg({'mp_price':['sum']})

    
    
        df_ag_adm2.month.astype(float)
        df_ag_adm2.mp_price.astype(float)
        df_ag_adm2['mp_price'] = df_ag_adm2['mp_price'].astype(float)
        
# to create a data frame with name of markets as columns and price of each product in each month  into every column
        df_ag_adm2_table = df_ag_adm2.pivot_table(values='mp_price', index= ['year', 'month','adm1_name','adm2_name','cm_name'], columns='market', aggfunc='sum')
        
#to aggregate 'df_ag_adm2_table' according to each market   
        market_name = df_ag_adm2.market.unique()
    
        df_ag_market = df_ag_adm2_table.groupby(['year', 'month','adm1_name','adm2_name', 'cm_name'])[market_name].sum()
    
      
        product= df_ag_market['cm_name'].unique()
        


 
 path =r"C:\Users\ASUS\Desktop\WHK\inflation\ " #inorder to save our files for some necessary steps I have written a path to save file after this process to desired folder   
    
        
#to calculate inflation for each product in each market     
    
    for u in product:
        
        
        
        filemo = path + u + i +'.csv' #to save data with name of country and name of adm2
        
        inf_product = df_ag_market[df_ag_market['cm_name']==u]
        
        
        
        inf_product['inflation'+' ' + market_name ] = (inf_product[market_name].diff()/inf_product[market_name])*100
        
       
        
        inf_product = inf_product.reset_index()
        
        
        inf_product.to_csv(filemo)




# Average inflation for each market

In [ ]:
# to calculate average amount of inflation for all products in each market



loc_pri = pd.read_csv('match_price_loc.csv') #loading('match_price_loc.csv')dataframe that has every market with its location and prices for each product(CSV file is available)



loc_pri_ad2 = loc_pri.admin2.unique()

df = pd.DataFrame()



path =r"C:\Users\ASUS\Desktop\prak\ " # create path for saving our data to desired destination


for file in os.listdir(os.getcwd()):
    for  adm2 in loc_pri_ad2:
        
        filee = path + adm2 + '.csv' # created to save our data with name of adm2
        
        if file.endswith( '.csv'):
            
            df = df.append(pd.read_csv(file))
            
            loc_price = loc_pri[loc_pri['admin2']== adm2]
            
            # to creat a dataframe in the format of saved inflation data we will create 't' as bellow:

            t = 'inflation' + ' '+ loc_price.market.unique()
            
            df  = df[df['adm2_name']== adm2 ]
            
            df2 = df.groupby(['year', 'month','adm2_name'])[t].mean()
            
            df2.to_csv(filee)

# Average inflation in each adm2

In [ ]:
for file in os.listdir(os.getcwd()):
    for  y in ad2:
        filee = path + y +' '+'inf_month'+ '.csv'#use this to save in desired folder with desired name 
        
        if file.endswith( y + '.csv'):
            
            dff = pd.read_csv(file)
   
            dff['mean'] = dff.iloc[:, 3:].mean(axis=1) #as in the created dataframes from forth column there will be markets and their prices so we need to start getting avg from it 
            
            
            
        
            dff.to_csv(filee) #now we have one dataframe as "dff" which has average amount of inflation in each admin2   

# Merging conflict data and inflation data

In [ ]:
# first, loading inflation data,  GED and ACLED data which intersected with gaul via ARcgis and all of them has HA0 and IPC code

ged = pd.read_csv('GED_CS_HA_inf_final.csv')
acled = pd.read_csv('ACLED_CS_HA_inf.csv')
inflation = pd.read_csv('all_admins_f.csv')

In [ ]:
#first, merge GED and inflation 

merg_ged = pd.merge(ged,inflation, on=['Year', 'Month', 'adm0','adm1','adm2','Fatalities_GED','number_event_GED'], how='left' )

In [ ]:
# then merge them with ACLED data
merg_ged_acled = pd.merge(acled,merg_ged, on=['Year', 'Month', 'adm0','adm1','adm2','Fatalities_GED','number_event_GED'], how='left' )